<a href="https://colab.research.google.com/github/wilzamguerrero/blendz_colab/blob/main/BLNDZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend1.png?raw=true' height="50" alt="netdata"/></center>
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
import os

download_path = "/content/sample_data" #@param {type: "string"}
blender_url = "https://download.blender.org/release/Blender4.2/blender-4.2.0-linux-x64.tar.xz" #@param {type: "string"}

print('\033[90m' + 'BLENDZ Installing Requirements 1% ✓' + '\033[0m')
with capture.capture_output() as cap:

  os.makedirs(download_path, exist_ok=True)
  !wget {blender_url} -P {download_path}
  os.chdir(download_path)

print('\033[90m' + 'BLENDZ Installing Requirements 30% ✓' + '\033[0m')
with capture.capture_output() as cap:

  !tar xf blender-*.tar.xz
  os.environ["LD_PRELOAD"] = ""
  !apt remove libtcmalloc-minimal4
  !apt install libtcmalloc-minimal4
  os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

print('\033[90m' + 'BLENDZ Installing Requirements 60% ✓' + '\033[0m')
with capture.capture_output() as cap:
  !sudo apt-get purge nvidia*
  !sudo add-apt-repository ppa:graphics-drivers/ppa -y
  !sudo apt-get update
  !sudo apt-get install expect -y

  expect_script = """
  spawn sudo apt install nvidia-driver-510 nvidia-dkms-510 -y
  expect "Configuring keyboard-configuration"
  send "32\r"
  expect "Country of origin for the keyboard"
  send "1\r"
  expect eof
  """
  with open("install_nvidia.exp", "w") as script_file:
      script_file.write(expect_script)

  !expect install_nvidia.exp
  os.remove("install_nvidia.exp")

  clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ Installation Complete 100% ✓' + '\033[0m')


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>

from IPython.display import clear_output, display, HTML
blender_path = "/content/sample_data/blender-4.2.0-linux-x64"  # @param {type: "string"}
filename = "/content/a2.blend"  # @param {type: "string"}
engine_option = "BLENDER_EEVEE_NEXT CUDA"  # @param ["CYCLES CUDA", "CYCLES OPTIX", "BLENDER_EEVEE_NEXT CUDA"]

start_frame = 105  # @param {type:"number"}
use_sequence = False  # @param {type:"boolean"}
end_frame = 107  # @param {type:"number"}

output_path = "/content/sample_data"  # @param {type: "string"}


if use_sequence:
    frame_option = f"-s {start_frame} -e {end_frame}"
else:
    frame_option = f"-s {start_frame} -e {start_frame}"


if engine_option == "CYCLES CUDA":
    !{blender_path}/blender -P "gpu.py" -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -- --cycles-device CUDA
elif engine_option == "BLENDER_EEVEE_NEXT CUDA":
    !{blender_path}/blender -P "gpu.py" -b {filename} -noaudio -E "BLENDER_EEVEE_NEXT" --debug-all -o "{output_path}/####" {frame_option} -a -- --BLENDER_EEVEE_NEXT-device CUDA
elif engine_option == "CYCLES OPTIX":
    !{blender_path}/blender -P "gpu.py" -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -- --cycles-device OPTIX

clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ Installation Complete 100% ✓' + '\033[0m')


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/install.png?raw=true' height="50" alt="netdata"/></center>
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
}
setInterval(ClickConnect,60000)
"""

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/compress.png?raw=true' height="50" alt="netdata"/></center>
import os, sys, re

folder_path = "/content/sample_data" #@param {type:"string"}
archiving_method = 'zip' #@param ["zip", "7z" ]

output_file_path = re.search("^[\/].+\/", folder_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", folder_path)
folder_name = re.sub("^[\/]", "", delsplit.group(0))

os.environ['inputDir'] = folder_path
os.environ['outputPath'] = output_file_path_raw
os.environ['folderName'] = folder_name

if archiving_method == 'zip':
   !7z a -tzip "$outputPath"/"$folderName".zip "$inputDir"

if archiving_method == '7z':
   !7z a -t7z -m0=lzma -mx=9 -mfb=258 -md=32m -ms=on "$outputPath"/"$folderName".7z "$inputDir"

In [ ]:
!/content/sample_data/blender-4.2.0-linux-x64/blender -P 'gpu.py' -b $filename -noaudio -E 'CYCLES' --debug-all -o "/content/sample_data/####" -s 105 -e 107 -a -F 'PNG' -- --cycles-device CUDA
!/content/sample_data/blender-4.2.0-linux-x64/blender -P 'gpu.py' -b $filename -noaudio -E 'BLENDER_EEVEE_NEXT' --debug-all -o "/content/sample_data/####" -s 1 -e 20 -a -F 'PNG' -- --BLENDER_EEVEE_NEXT-device CUDA
!/content/sample_data/blender-4.2.0-linux-x64/blender -P 'gpu.py' -b $filename -noaudio -E 'CYCLES' --debug-all -o "/content/sample_data/####" -s 108 -e 110 -a -F 'PNG' -- --cycles-device OPTIX

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>

blender_path = "/content/sample_data/blender-4.2.0-linux-x64"  # @param {type: "string"}
filename = "/content/a2.blend"  # @param {type: "string"}
engine_option = "CYCLES CUDA"  # @param ["CYCLES CUDA", "CYCLES OPTIX", "BLENDER_EEVEE_NEXT CUDA"]

start_frame = 109  # @param {type:"number"}
use_sequence = False  # @param {type:"boolean"}
end_frame = 112  # @param {type:"number"}

image_format = "PNG"  # @param ["PNG", "JPEG", "TIFF"]

output_path = "/content/sample_data"  # @param {type: "string"}

control_samples = True  # @param {type:"boolean"}
samples = 10  # @param {type:"number"}

if use_sequence:
    frame_option = f"-s {start_frame} -e {end_frame}"
else:
    frame_option = f"-s {start_frame}"

# Crear un script Python para configurar los samples
script_content = f"""
import bpy
bpy.context.scene.cycles.samples = {samples}
"""

with open("set_samples.py", "w") as script_file:
    script_file.write(script_content)

# Construir el comando para ejecutar Blender
if engine_option == "CYCLES CUDA":
    if control_samples:
        !{blender_path}/blender -b {filename} -P set_samples.py -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device CUDA
    else:
        !{blender_path}/blender -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device CUDA
elif engine_option == "BLENDER_EEVEE_NEXT CUDA":
    !{blender_path}/blender -b {filename} -noaudio -E "BLENDER_EEVEE_NEXT" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --BLENDER_EEVEE_NEXT-device CUDA
elif engine_option == "CYCLES OPTIX":
    if control_samples:
        !{blender_path}/blender -b {filename} -P set_samples.py -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device OPTIX
    else:
        !{blender_path}/blender -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device OPTIX

# Eliminar el script temporal
os.remove("set_samples.py")

clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ Render Complete 100% ✓' + '\033[0m')


In [ ]:
#@markdown progrss<left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>

blender_path = "/content/sample_data/blender-4.2.0-linux-x64"  # @param {type: "string"}
filename = "/content/a2.blend"  # @param {type: "string"}
engine_option = "CYCLES CUDA"  # @param ["CYCLES CUDA", "CYCLES OPTIX", "BLENDER_EEVEE_NEXT CUDA"]

start_frame = 130  # @param {type:"number"}
use_sequence = True  # @param {type:"boolean"}
end_frame = 150  # @param {type:"number"}

image_format = "PNG"  # @param ["PNG", "JPEG", "TIFF"]

output_path = "/content/sample_data"  # @param {type: "string"}

control_samples = True  # @param {type:"boolean"}
samples = 10  # @param {type:"number"}

if use_sequence:
    frame_option = f"-s {start_frame} -e {end_frame}"
else:
    frame_option = f"-s {start_frame} -e {start_frame}"

# Crear un script Python para configurar los samples
script_content = f"""
import bpy
bpy.context.scene.cycles.samples = {samples}
"""

with open("set_samples.py", "w") as script_file:
    script_file.write(script_content)

# Construir el comando para ejecutar Blender
if engine_option == "CYCLES CUDA":
    if control_samples:
        !{blender_path}/blender -b {filename} -P set_samples.py -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device CUDA
    else:
        !{blender_path}/blender -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device CUDA
elif engine_option == "BLENDER_EEVEE_NEXT CUDA":
    !{blender_path}/blender -b {filename} -noaudio -E "BLENDER_EEVEE_NEXT" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --BLENDER_EEVEE_NEXT-device CUDA
elif engine_option == "CYCLES OPTIX":
    if control_samples:
        !{blender_path}/blender -b {filename} -P set_samples.py -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device OPTIX
    else:
        !{blender_path}/blender -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device OPTIX

# Eliminar el script temporal
os.remove("set_samples.py")

clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ Render Complete 100% ✓' + '\033[0m')


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>
import subprocess
import sys
import gradio as gr
import io
import contextlib
import os
import re
from google.colab import drive
from IPython.display import clear_output, display, HTML
from IPython.utils import capture

def descargar_archivo(path):
    return path

def comprimir_archivo(folder_path, archiving_method):
    output_file_path = re.search("^[\/].+\/", folder_path)
    output_file_path_raw = output_file_path.group(0)
    delsplit = re.search("\/(?:.(?!\/))+$", folder_path)
    folder_name = re.sub("^[\/]", "", delsplit.group(0))

    os.environ['inputDir'] = folder_path
    os.environ['outputPath'] = output_file_path_raw
    os.environ['folderName'] = folder_name

    if archiving_method == 'zip':
        command = f"7z a -tzip '{output_file_path_raw}/{folder_name}.zip' '{folder_path}'"
    elif archiving_method == '7z':
        command = f"7z a -t7z -m0=lzma -mx=9 -mfb=258 -md=32m -ms=on '{output_file_path_raw}/{folder_name}.7z' '{folder_path}'"

    os.system(command)

    if archiving_method == 'zip':
        return f"{output_file_path_raw}/{folder_name}.zip"
    else:
        return f"{output_file_path_raw}/{folder_name}.7z"

def conectar_drive():
    drive.mount('/content/drive')
    return "Google Drive conectado en /content/drive"

def instalar_blender(download_path, blender_url):
    os.makedirs(download_path, exist_ok=True)
    !wget {blender_url} -P {download_path}
    os.chdir(download_path)

    !tar xf blender-*.tar.xz
    os.environ["LD_PRELOAD"] = ""
    !apt remove libtcmalloc-minimal4
    !apt install libtcmalloc-minimal4
    os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

    clear_output()
    return "BLENDZ Installation Complete 100% ✓"

with gr.Blocks() as iface:
    with gr.Tab("Descargar Archivo"):
        gr.Markdown("## Descargar un archivo")
        download_path = gr.Textbox(label="Ruta del Archivo", placeholder="Especifica la ruta del archivo")
        download_button = gr.Button("Descargar")
        download_output = gr.File(label="Descargar Archivo")
        download_button.click(descargar_archivo, inputs=download_path, outputs=download_output)

    with gr.Tab("Comprimir Archivo/Carpeta"):
        gr.Markdown("## Comprimir un archivo o carpeta")
        folder_path = gr.Textbox(label="Ruta de la Carpeta o Archivo", placeholder="/content/sample_data")
        archiving_method = gr.Dropdown(label="Método de Compresión", choices=["zip", "7z"], value="zip")
        compress_button = gr.Button("Comprimir")
        compress_output = gr.File(label="Archivo Comprimido")
        compress_button.click(comprimir_archivo, inputs=[folder_path, archiving_method], outputs=compress_output)

    with gr.Tab("Conectar a Google Drive"):
        gr.Markdown("## Conectar a Google Drive")
        drive_button = gr.Button("Conectar a Google Drive")
        drive_output = gr.Textbox(label="Estado de la conexión")
        drive_button.click(conectar_drive, outputs=drive_output)

    with gr.Tab("Instalar Blender"):
        gr.Markdown("## Instalar Blender")
        download_path = gr.Textbox(label="Ruta de Descarga", placeholder="/content/sample_data")
        blender_url = gr.Textbox(label="URL de Blender", value="https://download.blender.org/release/Blender4.2/blender-4.2.0-linux-x64.tar.xz")
        install_button = gr.Button("Instalar Blender")
        install_output = gr.Textbox(label="Estado de la instalación")
        install_button.click(instalar_blender, inputs=[download_path, blender_url], outputs=install_output)

output_buffer = io.StringIO()

with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
    iface.launch(debug=False, share=True)


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>
import subprocess
import sys
import io
import contextlib
import os
import re
from google.colab import drive
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
import requests

try:
    import gradio as gr
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio"])
    import gradio as gr

css_url = "https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/blendz.css"
response = requests.get(css_url)
css_content = response.text

def descargar_archivo(path):
    return path

def comprimir_archivo(folder_path, archiving_method):
    output_file_path = re.search("^[\/].+\/", folder_path)
    output_file_path_raw = output_file_path.group(0)
    delsplit = re.search("\/(?:.(?!\/))+$", folder_path)
    folder_name = re.sub("^[\/]", "", delsplit.group(0))

    os.environ['inputDir'] = folder_path
    os.environ['outputPath'] = output_file_path_raw
    os.environ['folderName'] = folder_name

    if archiving_method == 'zip':
        command = f"7z a -tzip '{output_file_path_raw}/{folder_name}.zip' '{folder_path}'"
    elif archiving_method == '7z':
        command = f"7z a -t7z -m0=lzma -mx=9 -mfb=258 -md=32m -ms=on '{output_file_path_raw}/{folder_name}.7z' '{folder_path}'"

    os.system(command)

    if archiving_method == 'zip':
        return f"{output_file_path_raw}/{folder_name}.zip"
    else:
        return f"{output_file_path_raw}/{folder_name}.7z"

def conectar_drive():
    drive.mount('/content/drive')
    return "Google Drive conectado en /content/drive"

def mostrar_auto_click_script():
    script = """
    function ClickConnect(){
        console.log("Working");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
    }
    setInterval(ClickConnect,60000)
    """
    return script

def instalar_blender(download_path, blender_url):
    os.makedirs(download_path, exist_ok=True)
    !wget {blender_url} -P {download_path}
    os.chdir(download_path)

    !tar xf blender-*.tar.xz
    os.environ["LD_PRELOAD"] = ""
    !apt remove libtcmalloc-minimal4
    !apt install libtcmalloc-minimal4
    os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"
    return "BLENDZ Installation Complete 100% ✓"

def fix_nvidia():

    !sudo apt-get purge nvidia* -y
    !sudo add-apt-repository ppa:graphics-drivers/ppa -y
    !sudo apt-get update
    !sudo apt-get install expect -y

    expect_script = """
    spawn sudo apt install nvidia-driver-510 nvidia-dkms-510 -y
    expect "Configuring keyboard-configuration"
    send "32\r"
    expect "Country of origin for the keyboard"
    send "1\r"
    expect eof
    """

    with open("install_nvidia.exp", "w") as script_file:
        script_file.write(expect_script)

    !expect install_nvidia.exp
    os.remove("install_nvidia.exp")

    return "NVIDIA fix completed successfully."

def renderizar_blender(blender_path, filename, engine_option, start_frame, use_sequence, end_frame, output_path):
    if use_sequence:
        frame_option = f"-s {start_frame} -e {end_frame}"
    else:
        frame_option = f"-s {start_frame} -e {start_frame}"

    if engine_option == "CYCLES CUDA":
        command = f"{blender_path}/blender -P gpu.py -b {filename} -noaudio -E CYCLES --debug-all -o {output_path}/#### {frame_option} -a -- --cycles-device CUDA"
    elif engine_option == "BLENDER_EEVEE_NEXT CUDA":
        command = f"{blender_path}/blender -P gpu.py -b {filename} -noaudio -E BLENDER_EEVEE_NEXT --debug-all -o {output_path}/#### {frame_option} -a -- --BLENDER_EEVEE_NEXT-device CUDA"
    elif engine_option == "CYCLES OPTIX":
        command = f"{blender_path}/blender -P gpu.py -b {filename} -noaudio -E CYCLES --debug-all -o {output_path}/#### {frame_option} -a -- --cycles-device OPTIX"

    os.system(command)
    return "Renderizado completado"

with gr.Blocks(css=css_content) as iface:
    gr.Markdown("# BLENDERZ")

    with gr.Tab("Instalar Blender"):
        gr.Markdown("## Instalar Blender")
        download_path = gr.Textbox(label="Ruta de Descarga", value="/content/sample_data")
        blender_url = gr.Textbox(label="URL de Blender", value="https://download.blender.org/release/Blender4.2/blender-4.2.0-linux-x64.tar.xz")
        install_button = gr.Button("Instalar Blender")
        install_output = gr.Textbox(label="Estado de la instalación")
        install_button.click(instalar_blender, inputs=[download_path, blender_url], outputs=install_output)

        gr.Markdown("## Fix de NVIDIA")
        fix_button = gr.Button("Aplicar Fix de NVIDIA")
        fix_output = gr.Textbox(label="Estado del Fix de NVIDIA")
        fix_button.click(fix_nvidia, outputs=fix_output)

    with gr.Tab("Renderizar Blender"):
        gr.Markdown("## Renderizar Blender")
        blender_path = gr.Textbox(label="Ruta de Blender", value="/content/sample_data/blender-4.2.0-linux-x64")
        filename = gr.Textbox(label="Archivo .blend", placeholder="Especifica la ruta del archivo")
        engine_option = gr.Dropdown(label="Motor de Renderizado", choices=["CYCLES CUDA", "CYCLES OPTIX", "BLENDER_EEVEE_NEXT CUDA"], value="CYCLES CUDA")
        start_frame = gr.Number(label="Frame de Inicio", value=1)
        use_sequence = gr.Checkbox(label="Usar Secuencia de Frames", value=False)
        end_frame = gr.Number(label="Frame de Finalización", value=24)
        output_path = gr.Textbox(label="Ruta de Salida", value="/content/sample_data")
        render_button = gr.Button("Renderizar")
        render_output = gr.Textbox(label="Estado del Renderizado")
        render_button.click(renderizar_blender, inputs=[blender_path, filename, engine_option, start_frame, use_sequence, end_frame, output_path], outputs=render_output)

    with gr.Tab("Conectar a Google Drive"):
        gr.Markdown("## Conectar a Google Drive")
        drive_button = gr.Button("Conectar a Google Drive")
        drive_output = gr.Textbox(label="Estado de la conexión")
        drive_button.click(conectar_drive, outputs=drive_output)

        gr.Markdown("## Ejecutar Auto Click en Colab")
        auto_click_script = gr.Textbox(value=mostrar_auto_click_script(), label="Código JavaScript para Auto Click", lines=10)
        gr.Markdown("Copia y pega este código en la consola de tu navegador para activar el auto click.")


    with gr.Tab("Comprimir Archivo/Carpeta"):
        gr.Markdown("## Comprimir un archivo o carpeta")
        folder_path = gr.Textbox(label="Ruta de la Carpeta o Archivo", placeholder="Especifica la ruta del archivo")
        archiving_method = gr.Dropdown(label="Método de Compresión", choices=["zip", "7z"], value="zip")
        compress_button = gr.Button("Comprimir")
        compress_output = gr.File(label="Archivo Comprimido")
        compress_button.click(comprimir_archivo, inputs=[folder_path, archiving_method], outputs=compress_output)

    with gr.Tab("Descargar Archivo"):
        gr.Markdown("## Descargar un archivo")
        download_path = gr.Textbox(label="Ruta del Archivo", placeholder="Especifica la ruta del archivo")
        download_button = gr.Button("Descargar")
        download_output = gr.File(label="Descargar Archivo")
        download_button.click(descargar_archivo, inputs=download_path, outputs=download_output)

output_buffer = io.StringIO()

with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
    iface.launch(debug=True, share=True)

#output = output_buffer.getvalue()
#for line in output.splitlines():
#    if "Running on public URL:" in line:
#        print(line.split("Running on public URL: ")[-1])
#        break

clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ FINISHED ✓' + '\033[0m')


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>
import subprocess
import sys
import io
import contextlib
import os
import re
from google.colab import drive
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
import requests

try:
    import gradio as gr
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio"])
    import gradio as gr

css_url = "https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/blendz.css"
response = requests.get(css_url)
css_content = response.text

def descargar_archivo(path):
    return path

def comprimir_archivo(folder_path, archiving_method):
    output_file_path = re.search("^[\/].+\/", folder_path)
    output_file_path_raw = output_file_path.group(0)
    delsplit = re.search("\/(?:.(?!\/))+$", folder_path)
    folder_name = re.sub("^[\/]", "", delsplit.group(0))

    os.environ['inputDir'] = folder_path
    os.environ['outputPath'] = output_file_path_raw
    os.environ['folderName'] = folder_name

    if archiving_method == 'zip':
        command = f"7z a -tzip '{output_file_path_raw}/{folder_name}.zip' '{folder_path}'"
    elif archiving_method == '7z':
        command = f"7z a -t7z -m0=lzma -mx=9 -mfb=258 -md=32m -ms=on '{output_file_path_raw}/{folder_name}.7z' '{folder_path}'"

    os.system(command)

    if archiving_method == 'zip':
        return f"{output_file_path_raw}/{folder_name}.zip"
    else:
        return f"{output_file_path_raw}/{folder_name}.7z"

def conectar_drive():
    drive.mount('/content/drive')
    return "Google Drive conectado en /content/drive"

def mostrar_auto_click_script():
    script = """
    function ClickConnect(){
        console.log("Working");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
    }
    setInterval(ClickConnect,60000)
    """
    return script

def instalar_blender(download_path, blender_url):
    os.makedirs(download_path, exist_ok=True)
    !wget {blender_url} -P {download_path}
    os.chdir(download_path)

    !tar xf blender-*.tar.xz
    os.environ["LD_PRELOAD"] = ""
    !apt remove libtcmalloc-minimal4
    !apt install libtcmalloc-minimal4
    os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"
    return "BLENDZ Installation Complete 100% ✓"

def fix_nvidia():

    !sudo apt-get purge nvidia* -y
    !sudo add-apt-repository ppa:graphics-drivers/ppa -y
    !sudo apt-get update
    !sudo apt-get install expect -y

    expect_script = """
    spawn sudo apt install nvidia-driver-510 nvidia-dkms-510 -y
    expect "Configuring keyboard-configuration"
    send "32\r"
    expect "Country of origin for the keyboard"
    send "1\r"
    expect eof
    """

    with open("install_nvidia.exp", "w") as script_file:
        script_file.write(expect_script)

    !expect install_nvidia.exp
    os.remove("install_nvidia.exp")

    return "NVIDIA fix completed successfully."

def renderizar_blender(blender_path, filename, engine_option, start_frame, use_sequence, end_frame, output_path):


    if use_sequence:
        frame_option = f"-s {start_frame} -e {end_frame}"
    else:
        frame_option = f"-s {start_frame} -e {start_frame}"

    if engine_option == "CYCLES CUDA":
        command = f"{blender_path}/blender -b {filename} -noaudio -E CYCLES --debug-all -o {output_path}/#### {frame_option} -a -- --cycles-device CUDA"
    elif engine_option == "CYCLES OPTIX":
        command = f"{blender_path}/blender -b {filename} -noaudio -E CYCLES --debug-all -o {output_path}/#### {frame_option} -a -- --cycles-device OPTIX"
    elif engine_option == "EEVEE_NEXT CUDA":
        command = f"{blender_path}/blender -b {filename} -noaudio -E BLENDER_EEVEE_NEXT --debug-all -o {output_path}/#### {frame_option} -a -- --BLENDER_EEVEE_NEXT-device CUDA"
    elif engine_option == "EEVEE_NEXT OPTIX":
        command = f"{blender_path}/blender -b {filename} -noaudio -E BLENDER_EEVEE_NEXT --debug-all -o {output_path}/#### {frame_option} -a -- --BLENDER_EEVEE_NEXT-device OPTIX"

    os.system(command)
    return "Renderizado completado"

with gr.Blocks(css=css_content) as iface:
    gr.Markdown("# BLENDERZ")

    with gr.Tab("Instalar Blender"):
        gr.Markdown("## Instalar Blender")
        download_path = gr.Textbox(label="Ruta de Descarga", value="/content/sample_data")
        blender_url = gr.Textbox(label="URL de Blender", value="https://download.blender.org/release/Blender4.2/blender-4.2.0-linux-x64.tar.xz")
        install_button = gr.Button("Instalar Blender")
        install_output = gr.Textbox(label="Estado de la instalación")
        install_button.click(instalar_blender, inputs=[download_path, blender_url], outputs=install_output)

        gr.Markdown("## Fix de NVIDIA")
        fix_button = gr.Button("Aplicar Fix de NVIDIA")
        fix_output = gr.Textbox(label="Estado del Fix de NVIDIA")
        fix_button.click(fix_nvidia, outputs=fix_output)

    with gr.Tab("Renderizar Blender"):
        gr.Markdown("## Renderizar Blender")
        blender_path = gr.Textbox(label="Ruta de Blender", value="/content/sample_data/blender-4.2.0-linux-x64")
        filename = gr.Textbox(label="Archivo .blend", placeholder="Especifica la ruta del archivo")
        engine_option = gr.Dropdown(label="Motor de Renderizado", choices=["CYCLES CUDA", "CYCLES OPTIX", "EEVEE_NEXT CUDA", "EEVEE_NEXT OPTIX"], value="CYCLES CUDA")
        start_frame = gr.Number(label="Frame de Inicio", value=1)
        use_sequence = gr.Checkbox(label="Usar Secuencia de Frames", value=False)
        end_frame = gr.Number(label="Frame de Finalización", value=24)
        output_path = gr.Textbox(label="Ruta de Salida", value="/content/sample_data")
        render_button = gr.Button("Renderizar")
        render_output = gr.Textbox(label="Estado del Renderizado")
        render_button.click(renderizar_blender, inputs=[blender_path, filename, engine_option, start_frame, use_sequence, end_frame, output_path], outputs=render_output)

    with gr.Tab("Conectar a Google Drive"):
        gr.Markdown("## Conectar a Google Drive")
        drive_button = gr.Button("Conectar a Google Drive")
        drive_output = gr.Textbox(label="Estado de la conexión")
        drive_button.click(conectar_drive, outputs=drive_output)

        gr.Markdown("## Ejecutar Auto Click en Colab")
        auto_click_script = gr.Textbox(value=mostrar_auto_click_script(), label="Código JavaScript para Auto Click", lines=10)
        gr.Markdown("Copia y pega este código en la consola de tu navegador para activar el auto click.")


    with gr.Tab("Comprimir Archivo/Carpeta"):
        gr.Markdown("## Comprimir un archivo o carpeta")
        folder_path = gr.Textbox(label="Ruta de la Carpeta o Archivo", placeholder="Especifica la ruta del archivo")
        archiving_method = gr.Dropdown(label="Método de Compresión", choices=["zip", "7z"], value="zip")
        compress_button = gr.Button("Comprimir")
        compress_output = gr.File(label="Archivo Comprimido")
        compress_button.click(comprimir_archivo, inputs=[folder_path, archiving_method], outputs=compress_output)

    with gr.Tab("Descargar Archivo"):
        gr.Markdown("## Descargar un archivo")
        download_path = gr.Textbox(label="Ruta del Archivo", placeholder="Especifica la ruta del archivo")
        download_button = gr.Button("Descargar")
        download_output = gr.File(label="Descargar Archivo")
        download_button.click(descargar_archivo, inputs=download_path, outputs=download_output)

output_buffer = io.StringIO()

with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
    iface.launch(debug=True, share=True)

#output = output_buffer.getvalue()
#for line in output.splitlines():
#    if "Running on public URL:" in line:
#        print(line.split("Running on public URL: ")[-1])
#        break

clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ FINISHED ✓' + '\033[0m')


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>
import subprocess
import sys
import io
import contextlib
import os
import re
from google.colab import drive
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
import requests
import signal

try:
    import gradio as gr
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio"])
    import gradio as gr

css_url = "https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/blendz.css"
response = requests.get(css_url)
css_content = response.text

render_process = None

def descargar_archivo(path):
    return path

def comprimir_archivo(folder_path, archiving_method):
    output_file_path = re.search("^[\/].+\/", folder_path)
    output_file_path_raw = output_file_path.group(0)
    delsplit = re.search("\/(?:.(?!\/))+$", folder_path)
    folder_name = re.sub("^[\/]", "", delsplit.group(0))

    os.environ['inputDir'] = folder_path
    os.environ['outputPath'] = output_file_path_raw
    os.environ['folderName'] = folder_name

    if archiving_method == 'zip':
        command = f"7z a -tzip '{output_file_path_raw}/{folder_name}.zip' '{folder_path}'"
    elif archiving_method == '7z':
        command = f"7z a -t7z -m0=lzma -mx=9 -mfb=258 -md=32m -ms=on '{output_file_path_raw}/{folder_name}.7z' '{folder_path}'"

    os.system(command)

    if archiving_method == 'zip':
        return f"{output_file_path_raw}/{folder_name}.zip"
    else:
        return f"{output_file_path_raw}/{folder_name}.7z"

def conectar_drive():
    drive.mount('/content/drive')
    return "Google Drive conectado en /content/drive"

def mostrar_auto_click_script():
    script = """
    function ClickConnect(){
        console.log("Working");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
    }
    setInterval(ClickConnect,60000)
    """
    return script

def instalar_blender(download_path, blender_url):
    os.makedirs(download_path, exist_ok=True)
    !wget {blender_url} -P {download_path}
    os.chdir(download_path)

    !tar xf blender-*.tar.xz
    os.environ["LD_PRELOAD"] = ""
    !apt remove libtcmalloc-minimal4
    !apt install libtcmalloc-minimal4
    os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"
    return "BLENDZ Installation Complete 100% ✓"

def fix_nvidia():

    !sudo apt-get purge nvidia* -y
    !sudo add-apt-repository ppa:graphics-drivers/ppa -y
    !sudo apt-get update
    !sudo apt-get install expect -y

    expect_script = """
    spawn sudo apt install nvidia-driver-510 nvidia-dkms-510 -y
    expect "Configuring keyboard-configuration"
    send "32\r"
    expect "Country of origin for the keyboard"
    send "1\r"
    expect eof
    """

    with open("install_nvidia.exp", "w") as script_file:
        script_file.write(expect_script)

    !expect install_nvidia.exp
    os.remove("install_nvidia.exp")

    return "NVIDIA fix completed successfully."

def renderizar_blender(blender_path, filename, engine_option, start_frame, use_sequence, end_frame, output_path):
    global render_process

    if use_sequence:
        frame_option = f"-s {start_frame} -e {end_frame}"
    else:
        frame_option = f"-s {start_frame} -e {start_frame}"

    if engine_option == "CYCLES CUDA":
        command = f"{blender_path}/blender -b {filename} -noaudio -E CYCLES --debug-all -o {output_path}/#### {frame_option} -a -- --cycles-device CUDA"
    elif engine_option == "CYCLES OPTIX":
        command = f"{blender_path}/blender -b {filename} -noaudio -E CYCLES --debug-all -o {output_path}/#### {frame_option} -a -- --cycles-device OPTIX"
    elif engine_option == "EEVEE_NEXT CUDA":
        command = f"{blender_path}/blender -b {filename} -noaudio -E BLENDER_EEVEE_NEXT --debug-all -o {output_path}/#### {frame_option} -a -- --BLENDER_EEVEE_NEXT-device CUDA"
    elif engine_option == "EEVEE_NEXT OPTIX":
        command = f"{blender_path}/blender -b {filename} -noaudio -E BLENDER_EEVEE_NEXT --debug-all -o {output_path}/#### {frame_option} -a -- --BLENDER_EEVEE_NEXT-device OPTIX"

    render_process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    output_lines = []
    for line in iter(render_process.stdout.readline, ''):
        output_lines.append(line.strip())
        yield "\n".join(output_lines)

    render_process.stdout.close()
    render_process.wait()
    return "Renderizado completado"

with gr.Blocks(css=css_content) as iface:
    gr.Markdown("# BLENDERZ")

    with gr.Tab("Instalar Blender"):
        gr.Markdown("## Instalar Blender")
        download_path = gr.Textbox(label="Ruta de Descarga", value="/content/sample_data")
        blender_url = gr.Textbox(label="URL de Blender", value="https://download.blender.org/release/Blender4.2/blender-4.2.0-linux-x64.tar.xz")
        install_button = gr.Button("Instalar Blender")
        install_output = gr.Textbox(label="Estado de la instalación")
        install_button.click(instalar_blender, inputs=[download_path, blender_url], outputs=install_output)

        gr.Markdown("## Fix de NVIDIA")
        fix_button = gr.Button("Aplicar Fix de NVIDIA")
        fix_output = gr.Textbox(label="Estado del Fix de NVIDIA")
        fix_button.click(fix_nvidia, outputs=fix_output)

    with gr.Tab("Renderizar Blender"):
        gr.Markdown("## Renderizar Blender")
        blender_path = gr.Textbox(label="Ruta de Blender", value="/content/sample_data/blender-4.2.0-linux-x64")
        filename = gr.Textbox(label="Archivo .blend", placeholder="Especifica la ruta del archivo")
        engine_option = gr.Dropdown(label="Motor de Renderizado", choices=["CYCLES CUDA", "CYCLES OPTIX", "EEVEE_NEXT CUDA", "EEVEE_NEXT OPTIX"], value="CYCLES CUDA")
        start_frame = gr.Number(label="Frame de Inicio", value=1)
        use_sequence = gr.Checkbox(label="Usar Secuencia de Frames", value=False)
        end_frame = gr.Number(label="Frame de Finalización", value=24)
        output_path = gr.Textbox(label="Ruta de Salida", value="/content/sample_data")
        render_button = gr.Button("Renderizar")
        render_output = gr.Textbox(label="Estado del Renderizado")
        render_button.click(renderizar_blender, inputs=[blender_path, filename, engine_option, start_frame, use_sequence, end_frame, output_path], outputs=render_output)

    with gr.Tab("Conectar a Google Drive"):
        gr.Markdown("## Conectar a Google Drive")
        drive_button = gr.Button("Conectar a Google Drive")
        drive_output = gr.Textbox(label="Estado de la conexión")
        drive_button.click(conectar_drive, outputs=drive_output)

        gr.Markdown("## Ejecutar Auto Click en Colab")
        auto_click_script = gr.Textbox(value=mostrar_auto_click_script(), label="Código JavaScript para Auto Click", lines=10)
        gr.Markdown("Copia y pega este código en la consola de tu navegador para activar el auto click.")


    with gr.Tab("Comprimir Archivo/Carpeta"):
        gr.Markdown("## Comprimir un archivo o carpeta")
        folder_path = gr.Textbox(label="Ruta de la Carpeta o Archivo", placeholder="Especifica la ruta del archivo")
        archiving_method = gr.Dropdown(label="Método de Compresión", choices=["zip", "7z"], value="zip")
        compress_button = gr.Button("Comprimir")
        compress_output = gr.File(label="Archivo Comprimido")
        compress_button.click(comprimir_archivo, inputs=[folder_path, archiving_method], outputs=compress_output)

    with gr.Tab("Descargar Archivo"):
        gr.Markdown("## Descargar un archivo")
        download_path = gr.Textbox(label="Ruta del Archivo", placeholder="Especifica la ruta del archivo")
        download_button = gr.Button("Descargar")
        download_output = gr.File(label="Descargar Archivo")
        download_button.click(descargar_archivo, inputs=download_path, outputs=download_output)

output_buffer = io.StringIO()

with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
    iface.launch(debug=True, share=True)

clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ FINISHED ✓' + '\033[0m')


In [ ]:
!apt install blender
!apt install libboost-all-dev
!apt install libgl1-mesa-dev